<a href="https://colab.research.google.com/github/mdi-group/mace-field-tutorial/blob/main/MACE_Field_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACE Field Architecture




In this tutorial, we dive into the alterations we have made to the MACE model to incorporate an external perturbing electric field into the MACE architecture, and how to use it to derive derivative properties such as the macroscopic polarisation, Born Effective Charges (BECS) and polarisability.

To learn how the base MACE code works, we highly recommend you look at the [MACE theory tutorial](https://colab.research.google.com/drive/1AlfjQETV_jZ0JQnV5M3FGwAM2SGCl2aU) developed by Will Baldwin and Ilyes Batatia.

In [ ]:
!git clone https://github.com/mdi-group/mace-field-tutorial.git
!pip install git+https://github.com/mdi-group/mace-field.git@field

In [ ]:
import torch
import numpy as np
from e3nn import o3
from matplotlib import pyplot as plt
import ase.io
import ase.data
from mace import data, modules, tools
from mace.tools import torch_geometric
torch.set_default_dtype(torch.float64)
import warnings
warnings.filterwarnings("ignore")
torch.manual_seed(3)

In [ ]:
from typing import Any, Dict, Optional
from mace.tools.scatter import scatter_sum
from mace.modules.blocks import (
    LinearReadoutBlock,
    ScaleShiftBlock,
)
from mace.modules.utils import (
    get_edge_vectors_and_lengths,
    get_symmetric_displacement,
)
from mace.modules.models import (
    MACE
)

In [ ]:
train_atoms = ase.io.read('data/ferroelectric_train_2040.xyz', ':')[0:1]
valid_atoms = ase.io.read('data/ferroelectric_valid.xyz', ':')[0:1]
train_configs = [data.config_from_atoms(atoms) for atoms in train_atoms]
valid_configs = [data.config_from_atoms(atoms) for atoms in valid_atoms]
atomic_numbers = []
for config in train_configs + valid_configs:
    atomic_numbers.extend(config.atomic_numbers)
atomic_numbers = list(set(atomic_numbers))
z_table = tools.AtomicNumberTable(atomic_numbers)
print(train_atoms)
print(valid_atoms)
print(z_table.zs)

\begin{equation}
    E_\alpha = E_\alpha^{(0)} + E_\alpha^{(1)} + \dots + E_\alpha^{(T)}
\end{equation}

\begin{equation}
  E_\alpha(t) = \mathcal{R}_t\left(\mathbf{h}_i^{(t)}\right) =
    \begin{cases}
    \begin{aligned}
        &\sum_{\tilde{k}} W_{\text{readout}, \tilde{k}}^{(t)} \left[ h_{\alpha,\tilde{k} 0 0}^{e, (t)} - \mathbf{p}_{\alpha, \tilde{k}}^{(t)} \cdot \mathbf{\mathcal{E}} \right] \qquad\ \text{if}\ t<T, \\
        &\text{MLP}_{\text{readout}}^{(t)}\left( \left\{ h_{\alpha,k 0 0}^{e, (t)} - \mathbf{p}_{\alpha, k}^{(t)} \cdot \mathbf{\mathcal{E}} \right\}_{k} \right) \quad \text{if}\ t = T.
    \end{aligned}
    \end{cases}
\end{equation}